# <span style = 'color: red'>***A.*** **Preprocesamiento**.

En este *notebook* se desarrolla el **preprocesamiento** de los datos, con la finalidad de obtener unos datos adecuados de cara al proceso de modelación. Así, se abordará la limpieza de datos, la transformación de los mismos, aplicación de reducción de dimensionalidad, balanceo de los datos y la construcción de conjuntos de entrenamiento y de prueba.

In [1]:
#
# Importanción de librerías de interés
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from janitor import clean_names
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import VarianceThreshold

# <span style = 'color: red'>***1.***</span> Lectura de los datos

Los datos se obtiene de Kaggle y están disponibles [aquí](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud/data). Es importante anotar que los dueños de la información reportan en la documentación de la misma (visible en el enlace anterior) que las características adjuntas son todas de corte numérico y ya pasaron por un preprocesamiento previo. Además, dada la sensibilidad, los dueños de la base establecen que no pueden revelar el significado de los diferentes campos, por lo que la mayoría de los datos están *anonimizados* de la forma $V_i, i = 1, \dots, 28$. Se tienen otras dos cantidad que no fueron transformadas: el tiempo (`time`, que representa la cantidad de segundos transcurridos entre la primera transacción del marco de datos y la del registro propiamente) y el monto de la transacción (`amount`). Sin embargo, como parte de un control de calidad, se realizará un preprocesamiento adicional.

Es clave anotar que la variable objetivo de esta base es `class` y que esta:
- es `1` cuando la transacción corresponde a un fraude.
- es `0` cuando la transacción *no* es fraudulenta.

In [ ]:
#
# Lectura de datos
direc = '../Datos/creditcard.csv.zip'
datos = pd.read_csv(direc, compression='zip')

# Limpieza de título
datos = datos.clean_names()

print(
    f'El conjunto de datos tiene {datos.shape[0]} filas y {datos.shape[1]} columnas.'
)

: 

In [ ]:
#
# Estructura de los datos
datos.info()

: 

In [ ]:
datos.head()

: 

De acuerdo con esta revisión inicial, y atendiendo la documentación de la base, la lectura de los datos es correcto y todas las características tienen la tipología de datos adecuada.

# <span style = 'color: red'>***2.***</span> Limpieza de los datos

## <span style = 'color: green'>**2.1.**</span> Duplicados y datos faltantes

Se comenzará revisando si existen filas duplicadas.

In [ ]:
#
# Revisión de duplicados
duplicados = datos.duplicated().sum()
print(f'El conjunto de datos tiene {duplicados} filas duplicadas.')

: 

Como se observa, para todas las características se tienen columnas de color gris continuo. Esto indica que la base carece de datos faltantes. De todos modos, se busca de manera manual si existen datos nulos, corroborando que no es el caso. Esto hace que no sea necesario imputar información y, por otra parte, el marco de datos será adecuado para cualquier tipo de modelo (entendiendo que algunos no aceptan valores faltantes).

In [ ]:
#
# Eliminación de duplicados
if duplicados > 0:
    datos = datos.drop_duplicates()
    print(f'Se eliminaron {duplicados} filas duplicadas.')
    print(f'El conjunto de datos ahora tiene {datos.shape[0]} filas y {datos.shape[1]} columnas.')
else:
    print('No se eliminaron filas duplicadas.')

: 

Ahora, se revisará la nulidad de información en esta base de datos:

In [ ]:
#
# Nulidad de información
msno.matrix(datos)
plt.title('Nulidad de información')
plt.show()

print(
    f'El conjunto de datos tiene {datos.isnull().sum().sum()} valores nulos.'
)

: 

## <span style = 'color: green'>**2.2.**</span> Observaciones atípicas

A continuación, se identificarán las observaciones atípicas, entendiéndolas como aquellas que están 1.5 veces el rango intercuartílico por encima del tercer cuartil ($Q3$) o por debajo del primer cuartil ($Q1$). Estos son los valores que típicamente quedan representados como puntos en gráficos de cajas y bigotes. Se realizará el análisis detallando si existe un sesgo especial hacia la clase que marca el fraude, lo que indica una relevancia especial de la característica.

In [ ]:
outlier_df = pd.DataFrame(index=datos.index)

for col in datos.select_dtypes(include=['number']).columns:
    q1 = datos[col].quantile(0.25)
    q3 = datos[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    outlier_df[col + '_outlier'] = ((datos[col] < lower) | (datos[col] > upper))


: 

In [ ]:
for col in datos.select_dtypes(include=['number']).columns:
    etiqueta_outlier = outlier_df[col + '_outlier']
    tabla = pd.crosstab(etiqueta_outlier, datos['class'], normalize='columns')  # proporción sobre cada clase
    print(f"\nDistribución de datos atípicos en {col}:")
    print(tabla)


: 

En general, los datos atípicos son importantes para poder identificar que una transacción tiene una mayor probabilidad de fraude. Esto se puede constatar con el hecho de que para las diferentes variables del modelo, los datos atípicos suelen distribuirse de manera más pareja o incluso modera a fuertemente sesgada para aquellas instancias en las que se marca un ***fraude***, caso contrario de las observaciones no fraudulentas, que suelen tener una menor asociación con los datos atípicos. 

Un ejemplo muy visible de esto ocurre con la característica `v14`: para esta, en las instancias fraudulentas, el 86.8 \% de ellas toman un valor atípico para esta variable, mientras que en las no fraudulentas solo el 4.8 \% de las observaciones serán atípicas. Es importante anotar que, si bien este comportamiento no se repite para todas las características, sí se observa algo semejante en la mayoría de ellas; en cambio, para las instancias no atípicas hay una baja tasa de datos atípicos.

Este análisis permite concluir que **no es adecuado eliminar, limpias o imputar las observaciones atípicas**.

## <span style = 'color: green'>**2.3.**</span> Análisis de multicolinealidad

Ahora, se revisará si existen características fuertemente asociadas entre sí mediante un análisis de multicolinealidad:

In [ ]:
#
# Matriz de correlación

plt.figure(figsize=(10, 8))
corr_matrix = datos.select_dtypes(include='number').corr()

plt.figure(figsize=(10,8))
plt.title("Matriz de correlación")
plt.imshow(corr_matrix, cmap='coolwarm', interpolation='nearest')
plt.colorbar()
plt.xticks(range(len(corr_matrix)), corr_matrix.columns, rotation=90)
plt.yticks(range(len(corr_matrix)), corr_matrix.columns)
plt.tight_layout()
plt.show()


: 

En general, las variables presentan una correlación lineal débil o moderada-débil entre sí. Solo vale la pena destacar la correlación entre `amount` y `v7`, que es de un $0.4$ aproximadamente, y la de `v2` y `amount`, que es de un $-0.6$ aproximadamente. Con esto en mente, no merece la pena eliminar ninguna variable del marco de datos entiendo como que puede quedar representada por otra muy altamente correlacionada.

Frente a la correlación con la variable objetivo, todas tienen correlaciones muy cercanas a cero. Las que más destacan son `v10`, `v14` y `v17`, con correlaciones de un $-0.1$ aproximadamente.

El anterior análisis se puede complementar con el $VIF$: *factor de inflación de varianza*. Valores de $VIF > 5$ pueden ser preocupantes.

In [ ]:
#
# VIF
X_vif = datos.select_dtypes(include='number').drop(columns=['target'], errors='ignore')  # omite target si es numérica

vif_data = pd.DataFrame()
vif_data["feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

print(vif_data)


: 

Como se observa, todas las variables tienen un $VIF < 3$, lo que las hace adecuadas para la modelación excepto por la columna del monto de la transacción con un $VIF_{\texttt{amount}} = 11.6$. No obstante, basados en un **criterio experto de negocio** (en riesgo de crédito), esta variable suele tener una incidencia importante en el fraude, por lo que vale la pena conservarla.  

## <span style = 'color: green'>**2.4.**</span> Estandarización

Como proceso de control de calidad de la información, y para evitar sesgos en el modelo, se va a realizar una estandarización de los datos